In [ ]:
import os
import re
from datetime import date
from dotenv import load_dotenv
import pandas as pd
import panel as pn
from sqlalchemy import create_engine, text

# Inicialização do Panel e variáveis de ambiente
pn.extension()
load_dotenv()

# Conexão com o PostgreSQL
engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)

# ===================================
#          ABA DE CONSULTA
# ===================================
filtro_nome   = pn.widgets.TextInput(name='Filtrar por Nome', placeholder='parte do nome...')
filtro_email  = pn.widgets.TextInput(name='Filtrar por Email', placeholder='parte do email...')
tipo_consulta = pn.widgets.RadioButtonGroup(name='Tipo', options=['Aluno','Professor'], button_type='primary')
btn_consultar = pn.widgets.Button(name='Consultar', button_type='success')
tabela_consulta = pn.pane.DataFrame(height=300)

def consulta(event):
    nome  = filtro_nome.value.strip()
    email = filtro_email.value.strip()
    if tipo_consulta.value == 'Aluno':
        df = pd.read_sql_query("""
            SELECT
              a.id_aluno AS id,
              p.pnome, p.sobrenome, p.email,
              a.matricula, a.datanascimento AS "dataNascimento"
            FROM aluno a
            JOIN pessoa p ON a.id_aluno = p.id_pessoa
        """, engine)
    else:
        df = pd.read_sql_query("""
            SELECT
              pr.id_professor AS id,
              p.pnome, p.sobrenome, p.email,
              COALESCE(STRING_AGG(DISTINCT f.curso, ', '), '') AS formacoes,
              COALESCE(STRING_AGG(DISTINCT ex.nome, ', '), '') AS experiencias
            FROM professor pr
            JOIN pessoa p ON pr.id_professor = p.id_pessoa
            LEFT JOIN formacao_professor fp ON pr.id_professor = fp.id_professor
            LEFT JOIN formacao f ON fp.id_formacao = f.id_formacao
            LEFT JOIN experiencia_professor ep ON pr.id_professor = ep.id_professor
            LEFT JOIN experiencia ex ON ep.id_experiencia = ex.id_experiencia
            GROUP BY pr.id_professor, p.pnome, p.sobrenome, p.email
        """, engine)
    mask = pd.Series(True, index=df.index)
    if nome:
        mask &= df['pnome'].str.contains(nome, case=False, na=False)
    if email:
        mask &= df['email'].str.contains(email, case=False, na=False)
    tabela_consulta.object = df[mask]

btn_consultar.on_click(consulta)

aba_consulta = pn.Column(
    "# Consulta de Aluno ou Professor",
    pn.Row(filtro_nome, filtro_email),
    tipo_consulta,
    btn_consultar,
    tabela_consulta
)

# ===================================
#         ABA DE INSERÇÃO
# ===================================
# Campos comuns
pnome     = pn.widgets.TextInput(name='Primeiro Nome', width=200)
sobrenome = pn.widgets.TextInput(name='Sobrenome', width=200)
email_in  = pn.widgets.TextInput(name='Email', width=300)
foto      = pn.widgets.TextInput(name='URL da Foto (opcional)', width=300)
bio       = pn.widgets.TextAreaInput(name='Bio (opcional)', height=60, width=500)
tipo_ins  = pn.widgets.RadioButtonGroup(name='Tipo', options=['Aluno','Professor'], button_type='primary')

# Painel Aluno
matricula   = pn.widgets.TextInput(name='Matrícula (6 dígitos)', width=200)
data_nasc   = pn.widgets.DatePicker(name='Data de Nascimento', width=200)
panel_aluno = pn.Column(pn.Row(matricula, data_nasc), visible=False)

# Listas auxiliares
lista_formacoes    = []
lista_experiencias = []

# Formação
novo_curso   = pn.widgets.TextInput(name='Curso', width=200)
novo_tempo   = pn.widgets.IntInput(name='Tempo (meses)', width=120)
novo_local   = pn.widgets.TextInput(name='Local', width=200)
novo_ira     = pn.widgets.FloatInput(name='IRA', width=80)
btn_add_form = pn.widgets.Button(name='Adicionar Formação', button_type='primary', width=180)

# Experiência
novo_exp_nome = pn.widgets.TextInput(name='Cargo', width=200)
novo_exp_dur  = pn.widgets.IntInput(name='Duração (meses)', width=120)
btn_add_exp   = pn.widgets.Button(name='Adicionar Experiência', button_type='primary', width=200)

# Painel Professor
panel_professor = pn.Column(
    pn.Row(novo_curso, novo_tempo, novo_local, novo_ira, btn_add_form),
    pn.Spacer(height=10),
    pn.Row(novo_exp_nome, novo_exp_dur, btn_add_exp),
    visible=False
)

# Alternância de painel
def toggle_tipo(event):
    panel_aluno.visible     = (tipo_ins.value == 'Aluno')
    panel_professor.visible = (tipo_ins.value == 'Professor')

toggle_tipo(None)
tipo_ins.param.watch(toggle_tipo, 'value')

# Mensagem de feedback
mensagem_ins = pn.pane.Markdown("")

# Botões de adicionar com validações
def adicionar_formacao(event):
    curso = novo_curso.value.strip()
    tempo = novo_tempo.value
    local = novo_local.value.strip()
    ira = novo_ira.value

    if ira < 7:
        mensagem_ins.object = "❌ O IRA da formação deve ser maior ou igual a 7. Formação não adicionada."
        return

    lista_formacoes.append({
        'Curso': curso,
        'Tempo': tempo,
        'Local': local,
        'IRA': ira
    })

    novo_curso.value = ''
    novo_tempo.value = 0
    novo_local.value = ''
    novo_ira.value = 0
    mensagem_ins.object = ""

btn_add_form.on_click(adicionar_formacao)

btn_add_exp.on_click(lambda e: lista_experiencias.append({
    'Cargo': novo_exp_nome.value.strip(),
    'Duração': novo_exp_dur.value
}) or (novo_exp_nome.param.update(value=''),
       novo_exp_dur.param.update(value=0)))

btn_inserir = pn.widgets.Button(name='Inserir', button_type='success', width=100)

def validar_email(email):
    return bool(re.match(r'^[^@]+@[^@]+\.[^@]+$', email))

def inserir(event):
    erros = []
    hoje = date.today()
    n, s, e = pnome.value.strip(), sobrenome.value.strip(), email_in.value.strip()

    if not n: erros.append("Nome é obrigatório.")
    if not s: erros.append("Sobrenome é obrigatório.")
    if not e: erros.append("Email é obrigatório.")
    elif not validar_email(e): erros.append("Email inválido.")

    if tipo_ins.value == 'Aluno':
        m, d = matricula.value.strip(), data_nasc.value
        if not re.fullmatch(r'\d{6}', m):
            erros.append("Matrícula deve ter exatamente 6 dígitos numéricos.")
        if not d:
            erros.append("Data de nascimento obrigatória.")
        elif (hoje - d).days/365.25 < 3:
            erros.append("Aluno deve ter pelo menos 3 anos de idade.")
    else:
        if len(lista_formacoes) < 1:
            erros.append("Professor precisa de ao menos 1 formação.")
        if len(lista_experiencias) < 1:
            erros.append("Professor precisa de ao menos 1 experiência.")

    if erros:
        mensagem_ins.object = "\n".join(f"❌ {msg}" for msg in erros)
        return

    with engine.begin() as conn:
        pid = conn.execute(text("""
            INSERT INTO pessoa(pnome, sobrenome, email, foto, bio)
            VALUES (:n, :s, :e, :f, :b)
            RETURNING id_pessoa
        """), {'n': n, 's': s, 'e': e, 'f': foto.value or None, 'b': bio.value or None}).scalar()

        if tipo_ins.value == 'Aluno':
            conn.execute(text("""
                INSERT INTO aluno(id_aluno, matricula, datanascimento)
                VALUES (:pid, :m, :d)
            """), {'pid': pid, 'm': matricula.value, 'd': data_nasc.value})
        else:
            conn.execute(text("INSERT INTO professor(id_professor) VALUES (:pid)"), {'pid': pid})
            for f in lista_formacoes:
                fid = conn.execute(text("""
                    INSERT INTO formacao(curso, tempo, local, ira)
                    VALUES (:c, :t, :l, :i)
                    RETURNING id_formacao
                """), {'c': f['Curso'], 't': f['Tempo'], 'l': f['Local'], 'i': f['IRA']}).scalar()
                conn.execute(text("""
                    INSERT INTO formacao_professor(id_professor, id_formacao)
                    VALUES (:pid, :fid)
                """), {'pid': pid, 'fid': fid})
            for x in lista_experiencias:
                eid = conn.execute(text("""
                    INSERT INTO experiencia(nome, duracao)
                    VALUES (:n, :d)
                    RETURNING id_experiencia
                """), {'n': x['Cargo'], 'd': x['Duração']}).scalar()
                conn.execute(text("""
                    INSERT INTO experiencia_professor(id_professor, id_experiencia)
                    VALUES (:pid, :eid)
                """), {'pid': pid, 'eid': eid})

        mensagem_ins.object = f"✅ {tipo_ins.value} inserido(a) com sucesso (id={pid})"
        lista_formacoes.clear()
        lista_experiencias.clear()

btn_inserir.on_click(inserir)

# Montar a aba final
aba_insercao = pn.Column(
    "# Inserção de Pessoa + Aluno/Professor",
    pn.Row(pnome, sobrenome, email_in, foto),
    bio,
    tipo_ins,
    panel_aluno,
    panel_professor,
    pn.Row(btn_inserir, mensagem_ins)
)

# ===================================
#         ABA DE REMOÇÃO
# ===================================
tipo_remocao   = pn.widgets.RadioButtonGroup(name='Tipo Remoção', options=['Aluno','Professor'], button_type='danger')
id_input       = pn.widgets.IntInput(name='ID', placeholder='Digite o ID', width=100)
btn_remover    = pn.widgets.Button(name='Remover', button_type='danger')
mensagem_remoc = pn.pane.Markdown("")

def remover(event):
    rid = id_input.value
    t   = tipo_remocao.value
    if not rid:
        mensagem_remoc.object = "❌ ID inválido."
        return
    with engine.begin() as conn:
        existe = conn.execute(
            text("SELECT 1 FROM pessoa WHERE id_pessoa = :rid"),
            {'rid': rid}
        ).fetchone()
    if not existe:
        mensagem_remoc.object = f"❌ ID {rid} inexistente."
        return
    try:
        with engine.begin() as conn:
            if t == 'Aluno':
                
                cids = conn.execute(text("""
                    SELECT id_certificado FROM historicoparticipacao WHERE id_aluno = :rid
                """), {'rid':rid}).fetchall()
                conn.execute(text("DELETE FROM historicoparticipacao WHERE id_aluno = :rid"), {'rid':rid})
                for (cid,) in cids:
                    conn.execute(text("DELETE FROM certificado WHERE id_certificado = :cid"), {'cid':cid})
               
                conn.execute(text("DELETE FROM favoritooficina WHERE id_aluno = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM favoritoprofessor WHERE id_aluno = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM listamatricula WHERE id_aluno = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM avaliacao WHERE id_aluno = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM aluno WHERE id_aluno = :rid"), {'rid':rid})
            else:
               
                conn.execute(text("DELETE FROM favoritoprofessor WHERE id_professor = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM formacao_professor WHERE id_professor = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM experiencia_professor WHERE id_professor = :rid"), {'rid':rid})
               
                oficinas = conn.execute(text("SELECT id_oficina FROM oficina WHERE id_professor = :rid"), {'rid':rid}).fetchall()
                for (ofid,) in oficinas:
                    conn.execute(text("DELETE FROM oficina_categorias      WHERE id_oficina = :ofid"), {'ofid':ofid})
                    conn.execute(text("DELETE FROM favoritooficina         WHERE id_oficina = :ofid"), {'ofid':ofid})
                    conn.execute(text("DELETE FROM listamatricula          WHERE id_oficina = :ofid"), {'ofid':ofid})
                    conn.execute(text("DELETE FROM historicoparticipacao   WHERE id_oficina = :ofid"), {'ofid':ofid})
                    conn.execute(text("DELETE FROM avaliacao              WHERE id_oficina = :ofid"), {'ofid':ofid})
                    conn.execute(text("DELETE FROM aula                   WHERE id_oficina = :ofid"), {'ofid':ofid})
                conn.execute(text("DELETE FROM oficina     WHERE id_professor = :rid"), {'rid':rid})
                conn.execute(text("DELETE FROM professor   WHERE id_professor = :rid"), {'rid':rid})
            conn.execute(text("DELETE FROM pessoa WHERE id_pessoa = :rid"), {'rid':rid})
        mensagem_remoc.object = f"✅ {t} com ID={rid} removido com sucesso."
    except Exception as e:
        mensagem_remoc.object = f"❌ Erro ao remover: {e}"

btn_remover.on_click(remover)

aba_remocao = pn.Column(
    "# Remoção por ID",
    tipo_remocao,
    id_input,
    btn_remover,
    mensagem_remoc
)

# Monta a aplicação e serve
app = pn.Tabs(
    ('Consulta', aba_consulta),
    ('Inserção', aba_insercao),
    ('Remoção', aba_remocao)
)
app.servable()
